In [ ]:
# @title Cell 1: Install Required Libraries & Setup
import subprocess
import os
from google.colab import files
import re

# Install ffmpeg (usually pre-installed in Colab, but ensures latest version)
!apt-get install -y ffmpeg

# Check installation
subprocess.run(["ffmpeg", "-version"])

print("✅ Setup Complete. Ready for processing.")

# Black & White

In [ ]:
# @title Cell 2: Black & White - Upload Video
from google.colab import files

print("📂 Please upload the video for the 'Black & White' task.")
uploaded = files.upload()

# Store the filename
bw_input_filename = list(uploaded.keys())[0]
print(f"✅ Uploaded: {bw_input_filename}")

In [ ]:
# @title Cell 3: Black & White - Process Video
import subprocess
import json

def get_video_info(filename):
    cmd = [
        'ffprobe', '-v', 'quiet', '-print_format', 'json',
        '-show_format', '-show_streams', filename
    ]
    result = subprocess.run(cmd, capture_output=True, text=True)
    return json.loads(result.stdout)

def replace_first_frame_black(input_file, output_file):
    print(f"⚙️ Processing {input_file}...")

    # 1. Get Metadata
    info = get_video_info(input_file)
    v_stream = info['streams'][0]
    fps = eval(v_stream['r_frame_rate']) # Convert fraction to float
    width = v_stream['width']
    height = v_stream['height']
    duration = float(info['format']['duration'])

    # Calculate duration of 1 frame
    frame_duration = 1 / fps

    print(f"📊 Detected: {width}x{height}, {fps:.2f} FPS")

    # 2. Create a 1-frame black video
    black_frame_file = "black_frame.mp4"
    cmd_black = [
        'ffmpeg', '-y', '-f', 'lavfi', '-i', f'color=c=black:s={width}x{height}:d={frame_duration}',
        '-c:v', 'libx264', '-pix_fmt', 'yuv420p', '-r', str(fps),
        '-t', str(frame_duration), black_frame_file
    ]
    subprocess.run(cmd_black, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    # 3. Trim original video (skip first frame, keep rest)
    rest_video_file = "rest_video.mp4"
    cmd_trim = [
        'ffmpeg', '-y', '-i', input_file,
        '-vf', f'trim=start_frame=1,setpts=PTS-STARTPTS',
        '-c:v', 'copy', '-an', rest_video_file
    ]
    subprocess.run(cmd_trim, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    # 4. Extract Audio
    audio_file = "audio.aac"
    cmd_audio = [
        'ffmpeg', '-y', '-i', input_file,
        '-q:a', '0', '-map', 'a', audio_file
    ]
    subprocess.run(cmd_audio, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    # 5. Concatenate Black Frame + Rest of Video + Audio
    # We must re-encode the video stream to join the generated black frame with the copied rest
    cmd_concat = [
        'ffmpeg', '-y',
        '-i', black_frame_file,
        '-i', rest_video_file,
        '-i', audio_file,
        '-filter_complex', '[0:v][1:v]concat=n=2:v=1:a=0[v]',
        '-map', '[v]', '-map', '2:a',
        '-c:v', 'libx264', '-crf', '18', '-preset', 'fast',
        '-c:a', 'copy',
        '-r', str(fps),
        output_file
    ]
    subprocess.run(cmd_concat, check=True)

    # Cleanup temps
    for f in [black_frame_file, rest_video_file, audio_file]:
        if os.path.exists(f): os.remove(f)

    print(f"✅ Saved as: {output_file}")

# Run the function
bw_output_filename = "output_black_white.mp4"
replace_first_frame_black(bw_input_filename, bw_output_filename)

# Grey

In [ ]:
# @title Cell 4: Grey - Upload Source & Target Videos
from google.colab import files

print("📂 Please upload 'video_source' and 'video_target'.")
uploaded = files.upload()

filenames = list(uploaded.keys())
if len(filenames) < 2:
    print("❌ Error: Please upload exactly 2 files.")
else:
    # Assign arbitrarily or ask user, here we assume order or name hint
    # For safety, we will print them and assign variables
    print(f"📄 Files found: {filenames}")
    # User should note which is which, but for code flow we assign:
    grey_source_filename = filenames[0]
    grey_target_filename = filenames[1]
    print(f"✅ Source: {grey_source_filename}")
    print(f"✅ Target: {grey_target_filename}")

In [ ]:
# @title Cell 5: Grey - Process Frame Swap
import subprocess
import json
import os

def get_frame_count(filename):
    cmd = [
        'ffprobe', '-v', 'error', '-select_streams', 'v:0',
        '-count_packets', '-show_entries', 'stream=nb_read_packets',
        '-of', 'csv=p=0', filename
    ]
    result = subprocess.run(cmd, capture_output=True, text=True)
    return int(result.stdout.strip())

def get_video_info(filename):
    cmd = [
        'ffprobe', '-v', 'quiet', '-print_format', 'json',
        '-show_format', '-show_streams', filename
    ]
    result = subprocess.run(cmd, capture_output=True, text=True)
    return json.loads(result.stdout)

def replace_first_frame_with_source_last(source_file, target_file, output_file):
    print("⚙️ Processing Grey Task...")

    # 1. Get Target Metadata
    target_info = get_video_info(target_file)
    t_stream = target_info['streams'][0]
    fps = eval(t_stream['r_frame_rate'])
    width = t_stream['width']
    height = t_stream['height']
    frame_duration = 1 / fps

    # 2. Get Source Frame Count to find "24th from last"
    total_frames = get_frame_count(source_file)
    target_frame_index = total_frames - 24

    if target_frame_index < 0:
        print(f"❌ Error: Source video has only {total_frames} frames. Need at least 24.")
        return

    print(f"📊 Source Frames: {total_frames} | Extracting Frame Index: {target_frame_index}")
    print(f"📊 Target Res: {width}x{height} | FPS: {fps:.2f}")

    # 3. Extract Specific Frame from Source
    extracted_frame = "extracted_frame.png"
    cmd_extract = [
        'ffmpeg', '-y', '-i', source_file,
        '-vf', f"select=eq(n\,{target_frame_index})",
        '-vframes', '1', extracted_frame
    ]
    subprocess.run(cmd_extract, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    # 4. Create 1-frame video from extracted image (matching target specs)
    new_first_frame = "new_first.mp4"
    cmd_img2vid = [
        'ffmpeg', '-y', '-loop', '1', '-i', extracted_frame,
        '-c:v', 'libx264', '-pix_fmt', 'yuv420p',
        '-t', str(frame_duration), '-r', str(fps),
        '-vf', f'scale={width}:{height}', new_first_frame
    ]
    subprocess.run(cmd_img2vid, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    # 5. Trim Target Video (Skip first frame)
    rest_target = "rest_target.mp4"
    cmd_trim = [
        'ffmpeg', '-y', '-i', target_file,
        '-vf', f'trim=start_frame=1,setpts=PTS-STARTPTS',
        '-c:v', 'copy', '-an', rest_target
    ]
    subprocess.run(cmd_trim, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    # 6. Extract Target Audio
    audio_file = "target_audio.aac"
    cmd_audio = [
        'ffmpeg', '-y', '-i', target_file,
        '-q:a', '0', '-map', 'a', audio_file
    ]
    subprocess.run(cmd_audio, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    # 7. Concatenate: New Frame + Rest of Target + Audio
    cmd_concat = [
        'ffmpeg', '-y',
        '-i', new_first_frame,
        '-i', rest_target,
        '-i', audio_file,
        '-filter_complex', '[0:v][1:v]concat=n=2:v=1:a=0[v]',
        '-map', '[v]', '-map', '2:a',
        '-c:v', 'libx264', '-crf', '18', '-preset', 'fast',
        '-c:a', 'copy',
        '-r', str(fps),
        output_file
    ]
    subprocess.run(cmd_concat, check=True)

    # Cleanup
    for f in [extracted_frame, new_first_frame, rest_target, audio_file]:
        if os.path.exists(f): os.remove(f)

    print(f"✅ Saved as: {output_file}")

# Run the function
grey_output_filename = "output_grey.mp4"
replace_first_frame_with_source_last(grey_source_filename, grey_target_filename, grey_output_filename)

# Download

In [ ]:
# @title Cell 6: Download All Processed Videos
import shutil
from google.colab import files

# List of output files to download
output_files = [
    "output_black_white.mp4",
    "output_grey.mp4"
]

# Filter files that actually exist (in case a previous cell failed)
existing_files = [f for f in output_files if os.path.exists(f)]

if not existing_files:
    print("❌ No output files found to download.")
else:
    print(f"📦 Preparing {len(existing_files)} files for download...")

    # Create a zip archive
    zip_name = "processed_videos.zip"
    shutil.make_archive("processed_videos", 'zip', '.', files=existing_files)

    # Trigger Download
    files.download(zip_name)
    print("✅ Download started!")